### PageRank algoritam

PageRank je algoritam na kojem je bazirano inicijalno rangiranje rezultata Guglove pretrage. U nekoj meri on se koristi i danas, uzimajući u obzir i druge aspekte stranica poput kvaliteta sadržaja, vremenske relevantnosti, personalizacije, raznovrsnosti ukupne liste rezultata i slično. <img src='assets/web.png'>

Pretpostavimo da je veb predstavljen usmerenim grafom u kojem su same stranice čvorovi grafa, a usmerena ivica od jednog ka drugom čvoru grafa hiperlink od jedne stranice ka drugoj, ukoliko postoji. Neka je skup svih čvorova označen sa $V$, skup svih čvorova iz kojih ne polazi nijedna ivica sa $V'$ (skup stranica koje nemaju linkove ka drugim stranicama). Neka su, dalje, za svaki čvor $v$, skupovi čvorova iz kojih vodi ivica ka $v$ i skupovi čvorova ka kojima polazi ivica iz $v$ označeni redom sa $P_v$ i $N_v$. Neka je $t_v$ numerička preferencija korisnika ka stranici $v$ takva da je zbir koordinata unutar vektora preferencija $t$ ka svim strnicama jednak $1$. Neka je još $n_v$ broj linkova na strani $v$ i $p_v$ verovatnoća da će korisnik posetiti stranicu $v$.

Kod pagerank algoritma se razmatraju dva faktora koja utiču na vrednost $p_v$. Sa jedne strane se razmatraju sve stranice $u \in P_v$ od kojih postoji hiperlink ka stranici $v$. Sa druge strane, pretpostavka je da se preko čvora $v$ hiperlinkom može stići do nekog čvora $u \in N_v \cap V'$ od koga ne postoji hiperlink ni ka jednoj stranici. Tada korisnik, sa nekom verovatnoćom usklađenom sa njegovim preferencijama može posetiti stranicu $v$. Dakle, za sve $v \in V$ važi

$$p_v = \sum_{u \in P_v} \frac{1}{n_u}p_u + \sum_{u \in N_v \cap V'} t_up_u.$$

Pokušajmo ovo da zapišemo u matričnom obliku. Neka je $k$ vektor takav da je $k_v = 1$ ako je $v \in V'$, a $k_v = 0$ inače. Neka je $A$ matrica takva da je $A_{ij} = 1/n_j$ ako postoji usmerena ivica od $j$ ka $i$, a $A_{ij} = 0$ inače. Tada je $p = (A + tk^T)p$.

Neka je $\alpha$ verovatnoća izbora jedne od veza na tekućoj strani. Tada je verovatnoća da se ne izabere veza na tekućoj strani jednaka $1 - \alpha$. Prethodni obrazac se za svako $v \in V$ sada  može preformulisati sa 

$$p_v = \sum_{u \in P_v} \frac{\alpha}{n_u}p_u + \sum_{u \in N_v \cap V'} \alpha t_up_u + (1 - \alpha) t_vp_v.$$

U matričnom obliku obrazac sada postaje $$p = (\alpha(A + tk^T) + (1 - \alpha)te^T)p$$ gde je $e$ vektor koji sadrži sve jedinice i čija je dužina jednaka dužini vektora $p$. 

Matrica $$G = \alpha(A + tk^T) + (1 - \alpha)te^T$$ se naziva `Guglova matrica`. Vektor $p$ je očigledno njen sopstveni vektor kojem odgovara sopstvena vrednost 1. Kako za matricu $G$ važi da je stohastička i da je vrednost 1 njena najveća sopstvena vrednost, $p$ će ujedno biti i njen najveći sopstveni vektor koji možemo pronaći metodom stepenovanja. 

U nastavku ćemo prikazati implementaciju `pagerank` algoritma na osnovu opisanog postupka. Nasumično ćemo generisati sve vrednosti vodeći računa o navedenim uslovima.

In [1]:
import numpy as np
from numpy import linalg as LA
np.random.seed(10)

Funkcija koja implementira metod stepenovanja. Iskoristićemo je za efikasno pronalaženje najveće sopstvene vrednosti Guglove matrice. 

In [2]:
def power_method(A, iterations):
    N = A.shape[0]
    b = np.random.rand(N)
    for _ in range(iterations):
        b = np.dot(A, b) / LA.norm(np.dot(A, b))
    return b

In [3]:
# Neka N predstavlja broj cvorova u grafu tj. velicinu veba
N = 10 

In [4]:
# Neka matrica A predstavlja matricu povezanosti izmedju cvorova-stranica
# Njeni element Aij ce imati vrednost 0 ukoliko ne postoji veza između cvora i i cvora j
# U suprotnom cemo elementima pridruziti nasumicne vrednosti ali tako da matrica bude stohasticka
A = np.zeros((N, N)) 

# Vektor k ce predstavljati vektor indeksa čvorova-stranica na kojima nema hiperveza 
k = np.zeros(N) 


# Generišimo nasumicno matricu A i na osnovu nje vektor k
def init(A, k, N):
    for i in range(0, N):

        # Generisemo proizvoljan  broj linkova na stranici
        number_of_links = np.random.randint(low=0, high=N)

        # Odredjujemo da li cvor treba upisati u niz k
        if number_of_links == 0:
            k[i] = 1

        # Generisemo kolonu matrice A koja sadrzi generisani broj linkova tj. jedinica i preostali broj nula 
        column = np.ones(number_of_links)
        column = np.append(column, np.zeros(N-number_of_links))
        np.random.shuffle(column)

        # Matrica treba da bude stohasticka matrica tj. sa zbirom 1 po kolonama
        if number_of_links != 0:
            column = column / number_of_links

        # Pridruzujemo kolonu matrici A
        A[:, i] = column

In [5]:
init(A, k, N)

In [6]:
A

array([[0.11111111, 0.        , 0.        , 0.33333333, 0.        ,
        0.        , 0.25      , 0.16666667, 0.        , 0.        ],
       [0.11111111, 0.        , 0.        , 0.33333333, 0.5       ,
        0.        , 0.25      , 0.16666667, 0.        , 0.        ],
       [0.11111111, 0.        , 0.16666667, 0.        , 0.        ,
        0.        , 0.25      , 0.        , 0.        , 0.        ],
       [0.11111111, 0.        , 0.16666667, 0.        , 0.        ,
        0.        , 0.        , 0.16666667, 0.        , 0.        ],
       [0.11111111, 0.        , 0.16666667, 0.        , 0.        ,
        0.5       , 0.        , 0.        , 0.        , 1.        ],
       [0.        , 0.        , 0.        , 0.33333333, 0.        ,
        0.        , 0.        , 0.16666667, 0.        , 0.        ],
       [0.11111111, 0.        , 0.16666667, 0.        , 0.5       ,
        0.        , 0.        , 0.16666667, 0.        , 0.        ],
       [0.11111111, 0.        , 0.1666666

In [7]:
k

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])

In [8]:
# parametar alpha cemo postaviti na originalnu vrednost PageRank algoritma
alpha = 0.85  

# Vektor e sadrzi sve jedinice
e = np.ones((N,1)) 

# Vektor preferencija t treba generisati proizvoljno, ali tako da mu je suma elemenata jednaka 1
t = np.random.rand(N)
t = t / np.sum(t)
t = t.reshape(N, 1)

# Prilagodicemo i dimenziju vektora k
k = k.reshape(N, 1)

In [9]:
# Odredjujemo matricu G
G = alpha*(A + np.dot(t, k.T)) + (1-alpha)*np.dot(t, e.T)

In [10]:
# Uverimo se da su sve vrednosti zbira kolona matrice G jednake 1
[np.sum(G[:, i]) for i in range(N)]

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999999999999, 1.0]

In [11]:
# Odredjujemo p kao sopstveni vektora matrice G koji odgovara najvecoj sopstvenoj vrednosti tj. 1
p = power_method(G, 1000)

In [12]:
p

array([0.19876382, 0.49299196, 0.15534342, 0.11883236, 0.43931351,
       0.11724512, 0.28317314, 0.188301  , 0.51868789, 0.29626841])

In [13]:
# Na osnovu vrednosti u vektoru p sada mozemo rangirati nas skup od N stranica
np.argsort(p)

array([5, 3, 2, 7, 0, 6, 9, 4, 1, 8])